In [216]:
# !pip install pandas-profiling

In [217]:
# !pip install spacy

In [89]:
# !pip install sentence_transformers

In [218]:
# !pip install en_core_web_sm-3.1.0-py3-none-any.whl

In [158]:
import spacy
import sklearn
import en_core_web_sm
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('bert-base-nli-mean-tokens')

import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import cosine_distances

nlp = spacy.load("en_core_web_sm")

In [159]:
import pandas as pd

In [160]:
df_combined_web_iva_search = pd.read_csv("s3://adl-core-sagemaker-studio/external/IVA/combined_new_adult-child_outer_new.csv")

/tmp/ipykernel_32824/1765538524.py:1: DtypeWarning: Columns (0,1,2,4,5,9,10,11,12,13,14,15,16,17,18,20,21,23,24,25,26,27,28,29,31,33,39,41,56,57,58,59,63,64,65,67,68,70,71,74,75,77,78,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,106,108,109,110,111,112,113,114,115,116,117,118,119,122,123,124,125,126,127,128,129,130,131,132,134,135,136,137,138,155,156,159,160,161,164,165) have mixed types. Specify dtype option on import or set low_memory=False.
  df_combined_web_iva_search = pd.read_csv("s3://adl-core-sagemaker-studio/external/IVA/combined_new_adult-child_outer_new.csv")


In [161]:
df_combined_web_iva_search['session_start_cst'] = pd.to_datetime(df_combined_web_iva_search['session_start_cst'])

In [162]:
df_combined_web_iva_search.shape


(1245878, 170)

In [163]:
df_combined_web_iva_search = df_combined_web_iva_search[['input','page_name','search_text', 'session_start_cst']]
# df_combined_web_iva_search.head(1)

In [164]:
df_combined_web_iva_search.drop_duplicates(inplace=True)

/tmp/ipykernel_32824/1427153576.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_combined_web_iva_search.drop_duplicates(inplace=True)


In [165]:
df_combined_web_iva_search.shape

(162161, 4)

In [166]:
import string
import pandas as pd

def clean_text(df, cols):
    """
    Clean the text in the specified columns of the DataFrame.
    Removes punctuation, white spaces, and converts text to lower case.
    
    Args:
    - df: pandas DataFrame with text columns to clean
    - cols: list of column names to clean
    
    Returns:
    - cleaned DataFrame
    """
    # Create a copy of the original DataFrame
    cleaned_df = df.copy()
    
    # Define a translation table to remove punctuation and special characters
    translator = str.maketrans("", "", string.punctuation + "{}[]/\|?><,.;:!@#+")
    
    # Loop through each column and clean the text
    for col in cols:
        # Convert text to lower case
        cleaned_df[col] = cleaned_df[col].fillna('').str.lower()
        # Remove punctuation, special characters, and white spaces
        cleaned_df[col] = cleaned_df[col].apply(lambda x: x.translate(translator).strip())
    
    return cleaned_df


In [167]:
df_combined_web_iva_search = clean_text(df_combined_web_iva_search, text_cols=['input','search_text','page_name'])

In [168]:
df_combined_web_iva_search.head(1)

,input,search_text,page_name
0,step child a dependent,nan,nan


In [169]:
df_combined_web_iva_search['text'] = df_combined_web_iva_search['input']+' '+df_combined_web_iva_search['search_text']+' '+df_combined_web_iva_search['page_name']

In [170]:
df_combined_web_iva_search.drop(['input','search_text','page_name'], inplace=True, axis=1)

In [171]:
df_combined_web_iva_search.drop_duplicates(subset=['text'], inplace=True)

In [172]:
df_combined_web_iva_search.shape

(123372, 1)

### finding the texts which contain exact phrases from synonyms list

In [173]:
words_1 = ['day care', 'creche', 'child care', 'childcare','daycare','nurseryschool','after school care',
           'pre school',
           'daynursery', 'infantschool','pre K', 'childcarer', 'childcarers',
 'infantcare', 'play school', 'playgroup',
 'nursery', 'nursery school', 'preschool', 'after schoolcare', 'day nursery', 'infant school', 
 'infant care', 'playschool', 'play group', 'kindergarten', 'childminding', 'babysitting', 'babysitter', 
 'nanny care', 'children supervision', 'toddler care', 'prekindergarten',
'child minding', 'baby sitting', 'baby sitter', 
 'nannycare', 'childrensupervision','child supervision', 'childsupervision', 'toddlercare', 'foster care', 'fostercare']
words_2 = [word.lower() for word in words_1]

In [174]:
mask = (df_combined_web_iva_search['text'].str.contains(r'\b(' + '|'.join(words_2) + r')\b', case=False, na=False))

df_combined_web_iva_search['category'] = ''
df_combined_web_iva_search.loc[mask, 'category'] = 'Child care'
# df_combined_web_iva_search.loc[df_combined_web_iva_search['category'] == '', 'category'] = 'Other'

df_combined_web_iva_search.head(5)

/tmp/ipykernel_32824/246165672.py:1: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  mask = (df_combined_web_iva_search['text'].str.contains(r'\b(' + '|'.join(words_2) + r')\b', case=False, na=False))


,text,category
0,step child a dependent nan nan,
1,how do i change my dependent daycare deduction...,Child care
2,child last name change nan nan,
3,i am trying to add my children as beneficiarie...,
4,how much timeoff do i get after the birth of m...,


### get text which are similar to phrases in synonnyms list for texts other than which are filtered above

In [175]:
def find_similar_sentences(sentences, phrases, threshold=0.95, category_name = 'Child care'):
    # encode the phrases using the model
    phrase_embeddings = model.encode(phrases, convert_to_tensor=True)
    
    # initialize an empty list to store the similar sentences
    similar_sentences = []
    
    # iterate over the sentences
    for sentence in sentences:
        # encode the sentence using the model
        sentence_embedding = model.encode(sentence, convert_to_tensor=True)
        # reshape the sentence embedding to a 2D array
        sentence_embedding = sentence_embedding.reshape(1, -1)
        
        # calculate the cosine similarity between the sentence embedding and each phrase embedding
        cosine_scores = 1 - sklearn.metrics.pairwise.cosine_distances(sentence_embedding, phrase_embeddings)
        
        # convert the cosine similarity scores to a list
        scores_list = cosine_scores.tolist()[0]
        
        # iterate over the phrases and similarity scores and append the sentence to the list if it meets the threshold for at least one phrase
        for phrase, score in zip(phrases, scores_list):
            if score >= threshold:
                similar_sentences.append(sentence)
                break
    
    # convert the list of similar sentences to a set to remove duplicates
    similar_sentences = set(similar_sentences)
    
    # create a new dataframe containing only the rows with text that is in the set of similar sentences
    similar_df = df_combined_web_iva_search[df_combined_web_iva_search['text'].isin(similar_sentences)]
    similar_df['category']=category_name
    return similar_df


In [176]:
df_combined_web_iva_search = df_combined_web_iva_search.dropna(subset=['text'])

In [177]:
similar_df = find_similar_sentences(df_combined_web_iva_search[df_combined_web_iva_search['category']=='']['text'].to_list(), 
                                    words_2)

In [178]:
similar_df

,text,category


In [179]:
only_CC_df = pd.concat([df_combined_web_iva_search[df_combined_web_iva_search['category']=='Child care'], 
                       similar_df]).sample(frac=1)

In [180]:
only_CC_df.shape

(33621, 2)

In [181]:
only_CC_df.drop_duplicates(inplace=True)

In [182]:
only_CC_df.shape

(33621, 2)

In [183]:
# filter out the rows with similar text from the original DataFrame
non_similar_df = df_combined_web_iva_search[~df_combined_web_iva_search['text'].isin(only_CC_df['text'])]

# sample twice as many rows from the non-similar DataFrame as there are in the similar DataFrame
non_similar_df = non_similar_df.sample(n=only_CC_df.shape[0])
non_similar_df['category'] = 'Other'

In [184]:
df_concatenated = pd.concat([only_CC_df, non_similar_df]).sample(frac=1).reset_index(drop=True)

In [185]:
df_concatenated.sample(5)

,text,category
1939,my child die nan nan,Other
53762,what is a day care reimbursement account nan nan,Child care
62319,how to enrol in child care child care hmcstmch...,Child care
60577,does the flexible spending account for depende...,Child care
9335,if contributing to a careaccount how often am ...,Child care


In [186]:
df_concatenated.shape

(67242, 2)

In [187]:
df_concatenated.category.value_counts(normalize=True), df_concatenated.category.value_counts()

(Child care    0.5
 Other         0.5
 Name: category, dtype: float64,
 Child care    33621
 Other         33621
 Name: category, dtype: int64)

In [188]:
df_concatenated.to_excel('CC_similar_non_similar_text_v4.xlsx')

### text preprocess

In [189]:
def count_intnt_entits(text):
    doc = nlp(text)
    intents = [token.text for token in doc if token.pos_ == 'VERB']
    entities = [token.text for token in doc if token.pos_ in {'NOUN', 'PROPN', 'ADJ', 'NUM', 'ADV'}]
    return len(intents), len(entities)

import re

def extract_ner_entities(sentence):
    doc = nlp(sentence)
    entities = [(entity.text, entity.label_) for entity in doc.ents]
    return entities

import numpy as np
def length_entities(list_entities):
    if (list_entities==np.nan or list_entities==None or list_entities==''):
        return 0
    else:
        return len(list_entities)
    
# filter_func = lambda x: any(item[1] in ["PERSON", "GPE", "ORG", "LOC", "FAC"] for item in x)
def filter_named_entities(text):
    # Process the text using Spacy
    doc = nlp(text)
    # Filter out named entities (ORG, PERSON, and GPE tags)
    filtered_words = [token.text for token in doc if token.ent_type_ not in ['ORG', 'PERSON', 'GPE', "LOC", "FAC"]]
    # Join the filtered words back into a string
    filtered_text = ' '.join(filtered_words)
    return filtered_text

In [190]:
import re
    
list_1 = ['ira','RMD','HRdirect','livechat','what is my hsa','P45','Payslip?',
    'sps','F80.2','ub','What is YSA','Paystub please','Sh','mfv','C-128','ax','no is hsa','FormL564','HIS','cif','GreT','YSACard',
    'Heli','RxPCN','403(b)','Hsa yes or no','ypr','Gv','ONA?','What is UHC?','HC-2','uo','what is 4DX?','osh','what is my hsa?',
    'sPRAVATO','sdr','RMD’s','coverage?How','This is for my hsa','pto?','A&DD','childcareplus','fs','mbi','Is that my lowesbenefit.com',
    'hra yes','mri?']
list_2 = [word.lower() for word in list_1]

def text_preprocess(dataframe, list_2):
    dataframe = dataframe.drop_duplicates()
    dataframe[['no_of_intents', 'no_of_entities']] = dataframe.apply(lambda x: pd.Series(count_intnt_entits(x['text'])), axis=1)  

    dataframe.loc[dataframe['text']!='', 'ner_enities'] = dataframe.loc[dataframe['text']!='', 'text'].apply(extract_ner_entities)
    dataframe['len_ner_enities'] = dataframe['ner_enities'].apply(length_entities)
    dataframe3 = dataframe[dataframe['len_ner_enities']>0]
    dataframe3['text'] = dataframe3['text'].apply(filter_named_entities)
    dataframe6 = pd.concat([dataframe[dataframe['len_ner_enities']==0], dataframe3], axis = 0)
    dataframe6 = dataframe6.drop(['no_of_intents','no_of_entities','ner_enities','len_ner_enities'], axis=1)

    dataframe6['text'] = dataframe6['text'].str.strip()
    
    return dataframe6

In [191]:
df_combined_cc_model_data_2 = text_preprocess(df_concatenated, list_2)

/tmp/ipykernel_32824/1699788337.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe3['text'] = dataframe3['text'].apply(filter_named_entities)


In [192]:
df_combined_cc_model_data_2.shape

(67242, 2)

In [193]:
df_combined_cc_model_data_3= df_combined_cc_model_data_2

In [194]:
df_combined_cc_model_data_3.columns

Index(['text', 'category'], dtype='object')

In [195]:
df_combined_cc_model_data_3['category'].value_counts()

Child care    33621
Other         33621
Name: category, dtype: int64

In [196]:
df_combined_cc_model_data_5 = df_combined_cc_model_data_3.copy()

## save unseen data for last few months

In [ ]:
monthly_counts_2 = df_combined_cc_model_data_5.resample('M', on='session_start_cst').count()

In [ ]:
monthly_counts_2

In [ ]:
df_combined_cc_model_data_5['session_start_cst'].max()

In [ ]:
# filterimg last six month data as unseen test data
start_date = pd.to_datetime('2023-02-26 00:00:00')
end_date = df_combined_cc_model_data_5['session_start_cst'].max()
unseen_test_df = df_combined_cc_model_data_5[(df_combined_cc_model_data_5['session_start_cst'] >= start_date) & (df_combined_cc_model_data_5['session_start_cst'] <= end_date)]

In [ ]:
unseen_test_df.shape

In [ ]:
unseen_test_df[['text', 'category']].to_excel('CC_unseen_data_v5.xlsx')

In [ ]:
final_CC_model_df = df_combined_cc_model_data_5[df_combined_cc_model_data_5['session_start_cst'] < start_date]

In [ ]:
final_CC_model_df['category'].value_counts()

In [ ]:
final_CC_model_df.to_excel('final_cc_model_data_v5.xlsx')

In [ ]:
final_CC_model_df.shape

# Load preprocessed data

In [ ]:
# Load the data
import pandas as pd
data = pd.read_excel('final_cc_model_data_v5.xlsx')
data.sample()

In [ ]:
data.sample()

In [ ]:
data.category.value_counts()

In [ ]:
data.category = data.category.map({'Other':0, 'Child care':1})
data.sample()

In [ ]:
data.info()

In [ ]:
data = data.dropna(subset=['text'])

In [ ]:
# data.text.value_counts(normalize=True)

In [ ]:
data.category.value_counts()

In [ ]:
data.to_excel('CC_texts_cols_preprocessed_data_v5.xlsx')

### Model building on the basis of just text columns

------

In [ ]:
import pandas as pd
import numpy as np
import torch
import os
from sklearn.metrics import accuracy_score, confusion_matrix

from torch.utils.data import DataLoader, TensorDataset, random_split
from transformers import BertForSequenceClassification, BertTokenizer, AdamW


# set the random seed for NumPy
np.random.seed(42)

# set the random seed for PyTorch
torch.manual_seed(42)

# load the data
# data = pd.read_csv('data.csv')

# load the BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [ ]:
data = data.dropna()

In [ ]:
data['category'] = data['category'].replace({'Child care':1, 'Other':0})

In [ ]:
data['category'].value_counts()

In [ ]:
data.info()

In [ ]:
import torch
import pandas as pd
import os
from sklearn.metrics import accuracy_score, confusion_matrix
from transformers import BertTokenizer, BertForSequenceClassification

# initialize the tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# encode the data using the tokenizer
encodings = tokenizer(list(data['text']),
                      truncation=True, padding=True)

# create the dataset
dataset = torch.utils.data.TensorDataset(
    torch.tensor(encodings['input_ids']),
    torch.tensor(encodings['attention_mask']),
    torch.tensor(list(data['category']))
)

# split the dataset into training and validation sets
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

# create the DataLoader for the training and validation sets
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=64, shuffle=False)

# load the BERT model for sequence classification
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2,)

# set up the optimizer and loss function
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)
loss_fn = torch.nn.CrossEntropyLoss()

In [ ]:
epochs = 10
start_epoch = 1

# set up checkpoint variables
best_accuracy = 0.0
epoch_losses = []
epoch_predictions = []
epoch_true_labels = []

In [ ]:
# train the model
for epoch in range(start_epoch, epochs):
    running_loss = 0.0
    for batch in train_loader:
        input_ids, attention_mask, labels = batch
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        logits = outputs.logits
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        running_loss += loss.item()
    print(f'Epoch {epoch + 1} Loss: {running_loss / len(train_loader)}')

    # make predictions on the data and compute the probabilities
    predictions = []
#     probabilities = []
    true_labels = []
    with torch.no_grad():
        model.eval()
        for batch in val_loader:
            input_ids, attention_mask, labels = batch
            outputs = model(input_ids, attention_mask=attention_mask)
            logits = outputs.logits
#             probs = torch.nn.functional.softmax(logits, dim=1)
            predictions.extend(torch.argmax(logits, dim=1).tolist())
#             probabilities.extend(probs[:, 1].tolist())  # probability of class 1 (assuming binary classification)
            true_labels.extend(labels.tolist())
    
    # convert probabilities to a NumPy array for easier handling
#     probabilities = np.array(probabilities)

    # print the probabilities for each row of data
#     print(f'Probabilities for each row of data:')
#     print(probabilities)
    
    accuracy = accuracy_score(true_labels, predictions)
    print(f'Epoch {epoch + 1} Validation Accuracy: {accuracy}')

    # generate the confusion matrix
    cm = confusion_matrix(true_labels, predictions)
    print(f'Epoch {epoch + 1} Confusion Matrix:')
    print(cm)
    
    # save the model's state and optimizer's state
    checkpoint = {
        'epoch': epoch + 1,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'loss': running_loss / len(train_loader)
    }
    torch.save(checkpoint, 'checkpoint1.pth')


In [ ]:
# load the checkpoint file
checkpoint = torch.load('checkpoint1.pth')

# extract the epoch, model's state and optimizer's state
start_epoch = checkpoint['epoch']
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
running_loss = checkpoint['loss']

# continue the training process from the last completed epoch
for epoch in range(start_epoch, epochs):
    running_loss = 0.0
    for batch in train_loader:
        input_ids, attention_mask, labels = batch
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        logits = outputs.logits
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        running_loss += loss.item()
    print(f'Epoch {epoch} Loss: {running_loss / len(train_loader)}')

    # make predictions on the data and compute the probabilities
    predictions = []
#     probabilities = []
    true_labels = []
    with torch.no_grad():
        model.eval()
        for batch in val_loader:
            input_ids, attention_mask, labels = batch
            outputs = model(input_ids, attention_mask=attention_mask)
            logits = outputs.logits
#             probs = torch.nn.functional.softmax(logits, dim=1)
            predictions.extend(torch.argmax(logits, dim=1).tolist())
#             probabilities.extend(probs[:, 1].tolist())  # probability of class 1 (assuming binary classification)
            true_labels.extend(labels.tolist())
    
    accuracy = accuracy_score(true_labels, predictions)
    print(f'Epoch {epoch} Validation Accuracy: {accuracy}')

    # generate the confusion matrix
    cm = confusion_matrix(true_labels, predictions)
    print(f'Epoch {epoch} Confusion Matrix:')
    print(cm)


In [137]:
train_size, val_size

(48757, 12190)

In [ ]:
# define the path to save the model
save_path = "./CC_model_outer_combined_texts_data_v4/"

# create the directory if it doesn't exist
import os
if not os.path.exists(save_path):
    os.makedirs(save_path)

# save the model
model.save_pretrained(save_path)


### convert saved model .bin and config.json to .pth compatible files

In [207]:
# convert saved model .bin and config.json to .pkl compatible files
from transformers import AutoConfig, AutoModelForSequenceClassification
import torch
import pickle

# Load the model configuration from the config.json file
config = AutoConfig.from_pretrained('CC_model_outer_combined_texts_data_v3', num_labels=2)

# Load the model from the binary file using the configuration
model = AutoModelForSequenceClassification.from_pretrained('CC_model_outer_combined_texts_data_v3', config=config)

# Save the model and configuration as a pickle file
with open('CC_model_outer_combined_texts_data_v3/CC_model.pth', 'wb') as f:
#     pickle.dump((config, model.state_dict()), f)
    torch.save((config, model.state_dict()), f)

## prediction code

In [ ]:
new_data = pd.read_excel('CC_unseen_data_v5.xlsx')[['text','category']]

In [ ]:
new_data.shape

In [ ]:
import pandas as pd
import torch
from transformers import BertTokenizer, BertForSequenceClassification

# load the BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# load the trained model
model = BertForSequenceClassification.from_pretrained(save_path)

# load the new data to be predicted
# new_data = pd.read_excel('EC_unseen_data.xlsx')

new_data['category'] = new_data['category'].replace({0:'Other', 1:'Child care'})

In [ ]:
new_data['category'].value_counts(dropna=False)

In [ ]:
new_data = new_data[new_data['text'].notna()]#.reset_index(drop=True)

In [ ]:
new_data.shape

In [ ]:
encodings = tokenizer(list(new_data['text']),
                      truncation=True, padding=True)

# create the dataset
dataset = torch.utils.data.TensorDataset(
    torch.tensor(encodings['input_ids']),
    torch.tensor(encodings['attention_mask'])
)

# create the DataLoader for the new data
loader = torch.utils.data.DataLoader(dataset, batch_size=32, shuffle=False)

# make predictions on the new data

predictions = []
probabilities = []
with torch.no_grad():
    model.eval()
    for batch in loader:
        input_ids, attention_mask = batch
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        probs = torch.nn.functional.softmax(logits, dim=1)
        probabilities.extend(probs[:, 1].tolist())
        predictions.extend(torch.argmax(logits, dim=1).tolist())

new_data['predictions'] = pd.Series(predictions).apply(lambda x: 'Child care' if x==1 else 'Other').values

new_data['probability'] = probabilities
# save the new data with predictions
new_data.to_csv('CC_predictions_v5.csv', index=False)

In [ ]:
new_data.info()

In [ ]:
pd.Series(predictions).apply(lambda x: 'Child care' if x==1 else 'Other').value_counts(dropna=False)

In [ ]:
new_data['category'].value_counts()

In [ ]:
new_data['predictions'].value_counts()

In [ ]:
# make predictions on the validation set and compute the accuracy
predictions = list(new_data['predictions'])
true_labels = list(new_data['category'])

accuracy = accuracy_score(true_labels, predictions)

# generate the confusion matrix
cm = confusion_matrix(true_labels, predictions)
print(cm)
print(accuracy)

In [ ]:
new_data.sample(5)

In [ ]:
new_data['predictions'].value_counts(), new_data['category'].value_counts()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt     

ax= plt.subplot()
sns.heatmap(cm, annot=True, fmt='g', ax=ax);  #annot=True to annotate cells, ftm='g' to disable scientific notation

# labels, title and ticks
accuracy = accuracy_score(true_labels, predictions)
print(accuracy)

ax.set_xlabel('Predicted labels');ax.set_ylabel('True labels'); 
ax.set_title('Confusion Matrix'); 
ax.xaxis.set_ticklabels(['Child care','Other']); ax.yaxis.set_ticklabels(['Child care', 'Other']);

In [ ]:
new_data[new_data['category']!=new_data['predictions']].to_csv('CC_model_wrong_predictions_v5.csv')

In [ ]:
new_data.to_csv('CC_model_all_predictions_v5.csv')